<a href="https://colab.research.google.com/github/Udayk7009/ImgHashtag/blob/main/ImageHashtagGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


**BLIP** **MODEL**





In [ ]:
import spacy
import en_core_web_sm
from itertools import chain
from nltk.corpus import wordnet as wn
import nltk
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

img_url = '/content/drive/MyDrive/Colab Notebooks/Image1.png' 
raw_image = Image.open(img_url).convert('RGB')

# image captioning
inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
genText = (processor.decode(out[0], skip_special_tokens=True))


selectNouns(genText)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['#association',
 '#football',
 '#participant',
 '#run',
 '#running',
 '#game',
 '#play']

In [ ]:
import requests
from PIL import Image
import torch

from transformers import OwlViTProcessor, OwlViTForObjectDetection

processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
texts = [["person"]]
inputs = processor(text=texts, images=image, return_tensors="pt")
outputs = model(**inputs)

# Target image sizes (height, width) to rescale box predictions [batch_size, 2]
target_sizes = torch.Tensor([image.size[::-1]])
# Convert outputs (bounding boxes and class logits) to COCO API
results = processor.post_process(outputs=outputs, target_sizes=target_sizes)

i = 0  # Retrieve predictions for the first image for the corresponding text queries
text = texts[i]
boxes, scores, labels = results[i]["boxes"], results[i]["scores"], results[i]["labels"]

# Print detected objects and rescaled box coordinates
score_threshold = 0.1
for box, score, label in zip(boxes, scores, labels):
    box = [round(i, 2) for i in box.tolist()]
    if score >= score_threshold:
        print(f"Detected {text[label]} with confidence {round(score.item(), 3)} at location {box}")


/usr/local/lib/python3.9/dist-packages/transformers/models/owlvit/image_processing_owlvit.py:355: FutureWarning: `post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`
  warnings.warn(


NLP **MODEL**

In [ ]:
def selectNouns(textInput):
  nlp = spacy.load("en_core_web_sm")
  nlp = en_core_web_sm.load()
  doc = nlp(textInput)
  nlpList =[(w.text, w.pos_) for w in doc]
  listOfWords = []
  for (i,j) in nlpList:
    if (j == 'NOUN' or j == 'VERB'):
      listOfWords.append(i)
  return genHashtags(listOfWords)


In [ ]:
print(selectNouns("arafed soccer player running on a soccer field with a ball sports person"))

['#association', '#football', '#participant', '#run', '#running', '#game', '#play', '#athletics', '#sport', '#individual', '#mortal', '#somebody', '#someone', '#soul']


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
doc = nlp("arafed soccer player running on a soccer field with a ball")


In [ ]:
list1 =[(w.text, w.pos_) for w in doc]
list1

[('arafed', 'VERB'),
 ('soccer', 'NOUN'),
 ('player', 'NOUN'),
 ('running', 'VERB'),
 ('on', 'ADP'),
 ('a', 'DET'),
 ('soccer', 'NOUN'),
 ('field', 'NOUN'),
 ('with', 'ADP'),
 ('a', 'DET'),
 ('ball', 'NOUN')]

In [ ]:
list2 = []
for (i,j) in list1:
    if j == 'NOUN':
        list2.append(i)
list2

['woman', 'beach', 'dog', 'cell', 'phone']

NLTK MODEL

In [ ]:
def genHashtags(inpText):
  hashtags = []
  for values in inpText:
    if wn.synonyms(values)== []:
      continue
    for value in wn.synonyms(values)[0]:
      hashtags.append(value) 
  return removeDuplicates(hashtags)

In [ ]:
def removeDuplicates(hashtags):
  #Seperating words joined with '_'
  hashtagSeperated = [item.split('_') for item in hashtags]
  #flattening list
  hashtag_flat = [item for l in hashtagSeperated for item in l]
  #Removing duplicates
  hashtags = list(dict.fromkeys(hashtag_flat))
  hashtags_final = ["#" + item for item in hashtags]
  return hashtags_final

In [ ]:
print(genHashtags(['arafed', 'soccer', 'player', 'running', 'soccer', 'field', 'ball']))

['#association', '#football', '#participant', '#run', '#running', '#game', '#play']


In [ ]:
print(genText)

arafed soccer player running on a soccer field with a ball [SEP]


In [ ]:
textInp = ['arafed', 'player', 'ball','field','football']
hashtags = []
for values in textInp:
  if wn.synonyms(values)== []:
    continue
  for value in wn.synonyms(values)[0]:
    hashtags.append(value) 
hashtags

['participant', 'football_game']

In [ ]:
x = [item.split('_') for item in hashtags]
x_flat = [item for l in x for item in l]
x_flat


['contact',
 'sport',
 'field',
 'game',
 'American',
 'football',
 'American',
 'football',
 'game',
 'professional',
 'football',
 'rugby',
 'rugby',
 'football',
 'rugger',
 'soccer',
 'association',
 'football',
 'ball']

In [ ]:
x_flat_new = ["#" + item for item in x_flat]
x_flat_new

['#contact',
 '#sport',
 '#field',
 '#game',
 '#American',
 '#football',
 '#American',
 '#football',
 '#game',
 '#professional',
 '#football',
 '#rugby',
 '#rugby',
 '#football',
 '#rugger',
 '#soccer',
 '#association',
 '#football',
 '#ball']

In [ ]:
%cd gdrive/MyDrive

[Errno 2] No such file or directory: 'gdrive/MyDrive'
/content


In [ ]:
#hashtags = []
for value in wn.synonyms("arafed")[0]:
  
    hashtags.append(value)

hashtags

IndexError: ignored

In [ ]:
wn.synonyms("running")[0]

['run', 'running_game', 'running_play']

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
l = []

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
WordNetLemmatizer().lemmatize("running", pos="n")

'running'

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk